In [12]:
### file to DB
# 1. CSV to table
# CSV 파일은 콤마를 기준으로 열단위 자료를 작성하는 파일로 데이터 베이스의 테이블 구조와 호환성이 높다.
# 1단계 : 테이블 생성 -> 레코드 추가 
# 2단계 : 

import pandas as pd
import pymysql

# 1. CSV파일을 읽기
bmi = pd.read_csv("data/bmi.csv")
bmi.info() 

'''
 0   height  20000 non-null  int64 
 1   weight  20000 non-null  int64 
 2   label   20000 non-null  object
'''

# 각 컬럼 추출
height = bmi['height']
weight = bmi['weight']
label = bmi['label']

# DB연동 환경 정보
config = {
    'host' : '127.0.0.1',  # 각자의 접속 DB주소, 127.0.0.1 
    'user' : 'scott',
    'password' : 'tiger',
    'database' : 'work',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True  # 유니코드를 안쓰면 아스키 코드를 쓰겠다는 의미이다. 가변길이
}

try:
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    # 테이블 확인(show tables)
    cursor.execute("show tables") 
    tables = cursor.fetchall() 
    
    # 스위칭 기법(2진 알고리즘) 
    sw = False
    for table in tables:
        if table[0] == 'bmi_tab': 
            sw = True # 테이블이 존재하는 경우
    # table실행 
    if not sw: # false라면 
        print("테이블 없음") # 테이블 생성
        sql = """create table bmi_tab(
            height int not null,
            weight int not null,
            label varchar(15) not null
        )"""  
        cursor.execute(sql)  
        
    # 레코드 조회
    cursor.execute("select * from bmi_tab")
    rows = cursor.fetchall() 
    if rows :  #레코드가 있는 경우
        for row in rows:
            print(f"{row[0]}\t,{row[1]}\t,{row[2]}") 
        print("전체 레코드 수 :", len(rows))
    else: # 레코드가 없는 경우
        print("100개 레코드 추가")
        for i in range(100): #0~99
            h = height[i]
            w = weight[i]
            lab = label[i] 
            cursor.execute(f"insert into bmi_tab values({h},{w},'{lab}')")
        conn.commit() 
except Exception as e:
    print("DB연동 에러 :",e)
    conn.rollback() 
    
finally:
    conn.close()
    cursor.close() 
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   height  20000 non-null  int64 
 1   weight  20000 non-null  int64 
 2   label   20000 non-null  object
dtypes: int64(2), object(1)
memory usage: 468.9+ KB
100개 레코드 추가


In [5]:
import pandas as pd
import pymysql

# 1. CSV파일을 읽기
score = pd.read_csv("data/score.csv")
score.info()
 
# 컬럼 추출
'''
 0   no      15 non-null     int64
 1   kor     15 non-null     int64
 2   eng     15 non-null     int64
 3   mat     15 non-null     int64
 4   dept    15 non-null     int64
''' 
no = score.no
kor = score.kor 
eng = score.eng 
mat = score.mat 
dept = score.dept

config = {
    'host' : '127.0.0.1',  # 각자의 접속 DB주소, 127.0.0.1 
    'user' : 'scott',
    'password' : 'tiger',
    'database' : 'work',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True   
}
# DB작업 및 table 생성 및 레코드 추가 
try:
    conn = pymysql.connect(**config) 
    cursor = conn.cursor() # db연결 객체 및 커서 객체 생성 
    cursor.execute("show tables") 
    tables = cursor.fetchall() 
    
    # 스위칭 기법(2진 알고리즘) 
    sw = False
    for table in tables:
        if table[0] == 'score_tab': 
            sw = True 
     # table실행 
    if not sw: # false가 아니라면  
        print("테이블 없음") # 테이블 생성 # create table if not exists score_tab(.... ) 
        sql = """create table score_tab(   
            no int primary key,
            kor int not null,
            eng int not null,
            mat int not null,
            dept int not null
        )"""  
        cursor.execute(sql)  
        
    cursor.execute("select * from score_tab")
    rows = cursor.fetchall() 
    
    if rows :  #레코드가 있는 경우
        print("번호\t국어\t영어\t수학\t반")
        for row in rows: 
            print(f"{row[0]}\t,{row[1]}\t,{row[2]}\t,{row[3]}\t,{row[4]}") 
            print("전체 레코드 수 :", len(rows))
    else: # 레코드가 없는 경우
         print("score.csv 파일을 DB에 레코드 추가")
    for i in range(len(score)):   
        n = no[i]
        k = kor[i]
        e = eng[i]
        m = mat[i]
        d = dept[i] 
        
        cursor.execute(f"insert into score_tab values({n},{k},{e},{m},{d})")
        conn.commit()

except Exception as e:
    print("DB연동 에러 :",e)
    conn.rollback() 
    
finally:
    conn.close()
    cursor.close() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   no      15 non-null     int64
 1   kor     15 non-null     int64
 2   eng     15 non-null     int64
 3   mat     15 non-null     int64
 4   dept    15 non-null     int64
dtypes: int64(5)
memory usage: 728.0 bytes
score.csv 파일을 DB에 레코드 추가


In [23]:
## json to table
# json은 {키:값} 형식으로 제공하는 파일 
# 1. json파일의 데이터를 디코딩 (딕셔너리로 변환해서 사용)
# 2. 데이터프레임 형식으로 변경하면 테이블 구조의 컬럼 단위 저장
import json 
import pandas as pd
import pymysql

# test = pd.read_json("data/labels.json")  # 간단한 방법
file = open("data/labels.json",'r',encoding='utf8') 
# 디코딩 : json-> object로 변환
lines = json.load(file)   # <class 'list'>
print(type(lines))
print(len(lines))         # 30개
print(type(lines[0]))     # <class 'dict'>

df = pd.DataFrame(lines)
print(df.info())
print(df.head())  

## db 작업 진행

config = {
    'host' : '127.0.0.1',  # 각자의 접속 DB주소, 127.0.0.1 
    'user' : 'scott',
    'password' : 'tiger',
    'database' : 'work',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True   
}

try: 
    conn = pymysql.connect(**config) 
    cursor = conn.cursor() # db연결 객체 및 커서 객체 생성 
    sql = """create table if not exists labels (
        id int not null,
        url varchar(150) not null,
        name varchar(50) not null,
        color varchar(50) not null,
        de char(5) not null
    )
    """
    cursor.execute(sql) 
    conn.commit() 
    
    # 레코드 조회
    cursor.execute("select * from labels")
    rows = cursor.fetchall() 
    if rows:
        print("labels 레코드 조회")
        for row in rows: 
            print(row)
        print("전체 레코드 수 :", len(rows))    
    else:  # 추가
        for i in range(len(df)):                                          
            uid = df.id[i]  #df.columns 
            url = df.url[i]
            name = df.name[i] 
            color = df.color[i]
            de = df.default[i]  
            
            sql = f"insert into labels values({uid},'{url}','{name}','{color}','{de}')" 
            cursor.execute(sql) 
            conn.commit()   
            
except Exception as e:
    print("DB연동 에러 :",e)
    conn.rollback() 
    
finally:
    conn.close()
    cursor.close() 




<class 'list'>
30
<class 'dict'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       30 non-null     int64 
 1   url      30 non-null     object
 2   name     30 non-null     object
 3   color    30 non-null     object
 4   default  30 non-null     bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 1.1+ KB
None
       id                                                url         name  \
0   76811  https://api.github.com/repos/pandas-dev/pandas...          Bug   
1   76812  https://api.github.com/repos/pandas-dev/pandas...  Enhancement   
2  127681  https://api.github.com/repos/pandas-dev/pandas...     Refactor   
3  129350  https://api.github.com/repos/pandas-dev/pandas...        Build   
4  134699  https://api.github.com/repos/pandas-dev/pandas...         Docs   

    color  default  
0  e10c02    False  
1  4E9A06    False  
2  FCE94F    Fal

In [64]:
import json 
import pandas as pd
import pymysql

# 저장된 json 파일을 불러와서 db작업을 하세요.

emp = pd.read_csv("data/emp.csv")
lst = [{'No':int(emp.No[i]),'Name':emp.Name[i],'Pay':int(emp.Pay[i])} for i in range(len(emp))]

# json.dumps(lst,eusure_ascii=False,indent=4) 
with open("data/emp.json","w",encoding='utf8') as f:
    jstr = json.dumps(lst,ensure_ascii=False,indent=4)
    f.write(jstr)
data = open("data/emp.json",'r',encoding="utf8")
data1 = json.load(data)  
df= pd.DataFrame(data1)  

config = {
    'host' : '127.0.0.1',  # 각자의 접속 DB주소, 127.0.0.1 
    'user' : 'scott',
    'password' : 'tiger',
    'database' : 'work',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True   
}

try: 
    conn = pymysql.connect(**config) 
    cursor = conn.cursor() # db연결 객체 및 커서 객체 생성 
    sql = """create table if not exists emp (
        no int not null,
        name varchar(50) not null,
        pay int(50) not null
    )
    """
    cursor.execute(sql) 
    conn.commit() 
    
    # 레코드 조회
    cursor.execute("select * from emp")
    rows = cursor.fetchall() 
    if rows:
        print("emp 레코드 조회")
        for row in rows: 
            print(row)
        print("전체 레코드 수 :", len(rows))    
    else:  # 추가
        for i in range(len(df)):                                          
            num = df.No[i]  #df.columns 
            name = df.Name[i]
            pay = df.Pay[i]    
            sql = f"insert into emp values({num},'{name}',{pay})"  
            cursor.execute(sql) 
            conn.commit()   
            
except Exception as e:
    print("DB연동 에러 :",e)
    conn.rollback() 
    
finally:
    conn.close()
    cursor.close() 




